In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [ ]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
Y_train_full = train_df['label']
X_train_full = train_df.drop(labels=['label'], axis=1)
Y_train_full.head()


In [ ]:
X_train_full.head()

# normalizing pixel values in X_train_full to 0 and 1 as it will be easier to train 

In [ ]:
X_train_full = X_train_full / 255.0
test_df = test_df / 255.0

**Reshaping the data from N,784 to N,28,28,1 for CNN**

In [ ]:
X_train_full = X_train_full.values.reshape(-1, 28, 28, 1)
X_test = test_df.values.reshape(-1, 28, 28, 1)

**One-Hot Encoding labels**

In [ ]:
Y_train_full = keras.utils.to_categorical(Y_train_full, num_classes=10)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train_full, Y_train_full, test_size=0.1, random_state=42)

In [ ]:
# Plot the first training image
plt.imshow(X_train[0][:,:,0], cmap='gray')
plt.title(f"Label: {np.argmax(Y_train[0])}")
plt.show()

# the reshape is working correctly

In [ ]:
model = keras.Sequential([
    # Explicit Input Layer to fix the warning
    layers.Input(shape=(28, 28, 1)),

    # First Convolutional Block
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    # Second Convolutional Block
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    # Flatten and Dense Layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=10, 
                    batch_size=64,
                    validation_data=(X_val, Y_val),
                    verbose=1)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
results = np.argmax(predictions, axis=1)

In [ ]:
submission = pd.DataFrame({
    "ImageId": range(1, len(results) + 1),
    "Label": results
})

In [ ]:
submission.to_csv("submission.csv", index=False)